In [11]:
# 🔹 2. Importar bibliotecas necessárias
from openai import OpenAI
import os
import pandas as pd

load_dotenv()
# 🔹 3. Configurar a chave de API
# Defina sua chave de API como variável de ambiente ou diretamente aqui (não recomendado em notebooks públicos)
# os.environ["OPENAI_API_KEY"] = "sk-xxxx"
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [52]:
# 🔹 4. Função para enviar prompt e capturar logprobs
def ask_gpt_with_logprobs(prompt: str, model: str = "gpt-4o-mini", top_logprobs: int = 5):
    """
    Envia um prompt ao modelo GPT e retorna:
      - resposta gerada
      - DataFrame com tokens e logprobs
    """
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        logprobs=True,
        top_p=0.9,
        top_logprobs=top_logprobs,
        max_tokens=10
    )

    # Extrair texto principal
    message = response.choices[0].message.content

    # Acessar tokens com logprobs
    token_data = response.choices[0].logprobs.content  # lista de ChatCompletionTokenLogprob

    # Extrair informações relevantes
    tokens, logprobs, top_alternatives = [], [], []
    for t in token_data:
        tokens.append(t.token)
        logprobs.append(t.logprob)
        # Salva as top alternativas para análise posterior
        alt = [(alt.token, alt.logprob) for alt in t.top_logprobs]
        top_alternatives.append(alt)

    # Criar tabela
    df = pd.DataFrame({
        "Token": tokens,
        "LogProb": logprobs,
        "Top Alternatives": top_alternatives
    })

    return message, df


In [ ]:
# 🔹 5. Teste da função
prompt = "Complete a frase: O rato roeu a roupa do rei de "
# prompt = "Complete a frase: Deus ajuda quem "
resposta, tabela_logprobs = ask_gpt_with_logprobs(prompt)


# 🔹 6. Exibir resultados
print("🧠 Resposta do GPT:\n", resposta)
print("\n📊 Tokens e LogProbs:\n")
display(tabela_logprobs)

🧠 Resposta do GPT:
 "Deus ajuda quem se ajuda." Essa expressão

📊 Tokens e LogProbs:



,Token,LogProb,Top Alternatives
0,"""",-1.077085,"[(De, -0.45208466053009033), ("", -1.0770846605..."
1,De,-0.000230,"[(De, -0.00022964231902733445), (Aj, -9.000229..."
2,us,0.000000,"[(us, 0.0), (US, -22.75), ( Deus, -23.5), (uz,..."
3,ajuda,-0.000005,"[( ajuda, -4.723352049040841e-06), ( ayuda, -1..."
4,quem,-0.000002,"[( quem, -1.7432603272027336e-06), ( quien, -1..."
5,se,-1.809440,"[( cedo, -0.18443959951400757), ( se, -1.80943..."
6,ajuda,-0.001117,"[( ajuda, -0.0011174732353538275), ( esfor, -7..."
7,".""",-0.000266,"[(."", -0.00026646157493814826), (""., -8.625266..."
8,Essa,-0.436669,"[( Essa, -0.4366685152053833), (<|end|>, -1.18..."
9,expressão,-0.257343,"[( expressão, -0.257343053817749), ( é, -2.007..."


In [ ]:
!pip install matplotlib

In [61]:
# 🔹 7. Visualizar probabilidades alternativas de tokens
import math
import matplotlib.pyplot as plt

def plot_token_alternatives(df, token_index=0):
    """
    Mostra um gráfico de barras com as top alternativas de token e suas probabilidades reais (exp(logprob)).
    """
    if token_index >= len(df):
        print("Índice fora do intervalo. Escolha entre 0 e", len(df)-1)
        return
    
    token_info = df.iloc[token_index]
    alternatives = token_info["Top Alternatives"]
    
    tokens = [t[0] for t in alternatives]
    probs = [math.exp(t[1]) for t in alternatives]  # converte logprobs para probabilidade real
    probs = [p / sum(probs) for p in probs]  # normaliza para somar 1

    plt.figure(figsize=(8, 4))
    plt.bar(tokens, probs)
    plt.title(f"Distribuição de Probabilidades - Token {token_index}: '{token_info['Token']}'")
    plt.ylabel("Probabilidade normalizada")
    plt.xlabel("Alternativas de tokens")
    plt.show()



In [ ]:
# Exemplo: exibir o gráfico das alternativas do 5º token
plot_token_alternatives(tabela_logprobs, token_index=6)